# Homework 7 - Network Compression (Network Pruning)

> Author: Arvin Liu (b05902127@ntu.edu.tw)

In [ ]:
folder_path = './'

In [ ]:
# Download dataset
!gdown --id '1GzukFVznTp_RG7b2ury7hr9TwA-MyMYj' --output food-11.zip
# Unzip the files
!unzip food-11.zip

In [ ]:
import random
import numpy as np
import torch

# 固定隨機種子
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(0)

# Readme


HW7的任務是模型壓縮 - Neural Network Compression。

Compression有很多種門派，在這裡我們會介紹上課出現過的其中四種，分別是:

* 知識蒸餾 Knowledge Distillation
* 網路剪枝 Network Pruning
* 用少量參數來做CNN Architecture Design
* 參數量化 Weight Quantization

在這個notebook中我們會介紹Network Pruning，
而我們有提供已經做完Knowledge Distillation的小model來做Pruning。

* Model架構 / Architecute Design在同目錄中的hw7_Architecture_Design.ipynb。
* 下載已經train好的小model(0.99M): https://drive.google.com/open?id=12wtIa0WVRcpboQzhgRUJOpcXe23tgWUL
  * 參數為 base=16, width_mult=1 (default)

In [ ]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
# # Load進我們的Model架構(在hw7_Architecture_Design.ipynb內)
# !gdown --id '1lJS0ApIyi7eZ2b3GMyGxjPShI8jXM2UC' --output "hw7_Architecture_Design.ipynb"
# %run "hw7_Architecture_Design.ipynb"

Network Pruning
===
在這裡我們會教Neuron Pruning。
<img src="https://i.imgur.com/Iwp90Wp.png" width="500px">

簡單上來說就是讓一個已經學完的model中的neuron進行刪減，讓整個網路變得更瘦。

## Weight & Neuron Pruning
* weight和neuron pruning差別在於prune掉一個neuron就等於是把一個matrix的整個column全部砍掉。但如此一來速度就會比較快。因為neuron pruning後matrix整體變小，但weight pruning大小不變，只是有很多空洞。

## What to Prune?
* 既然要Neuron Pruning，那就必須要先衡量Neuron的重要性。衡量完所有的Neuron後，就可以把比較不重要的Neuron刪減掉。
* 在這裡我們介紹一個很簡單可以衡量Neuron重要性的方法 - 就是看batchnorm layer的$\gamma$因子來決定neuron的重要性。 (by paper - Network Slimming)
  ![](https://i.imgur.com/JVpCm2r.png)
* 相信大家看這個pytorch提供的batchnorm公式應該就可以意識到為甚麼$\gamma$可以當作重要性來衡量了:)

* Network Slimming其實步驟沒有這麼簡單，有興趣的同學可以check以下連結。[Netowrk Slimming](https://arxiv.org/abs/1708.06519)


## 為甚麼這會 work?
* 樹多必有枯枝，有些neuron只是在躺分，所以有他沒他沒有差。
* 困難的說可以回想起老師說過的大樂透假說(The Lottery Ticket Hypothesis)就可以知道了。

## 要怎麼實作?
* 為了避免複雜的操作，我們會將StudentNet(width_mult=$\alpha$)的neuron經過篩選後移植到StudentNet(width_mult=$\beta$)。($\alpha > \beta$)
* 篩選的方法也很簡單，只需要抓出每一個block的batchnorm的$\gamma$即可。

## 一些實作細節
* 假設model中間兩層是這樣的:

|Layer|Output # of Channels|
|-|-|
|Input|in_chs|
|Depthwise(in_chs)|in_chs|
|BatchNorm(in_chs)|in_chs|
|Pointwise(in_chs, **mid_chs**)|**mid_chs**|
|**Depthwise(mid_chs)**|**mid_chs**|
|**BatchNorm(mid_chs)**|**mid_chs**|
|Pointwise(**mid_chs**, out_chs)|out_chs|

則你會發現利用第二個BatchNorm來做篩選的時候，跟他的Neuron有直接關係的是該層的Depthwise&Pointwise以及上層的Pointwise。
因此再做neuron篩選時記得要將這四個(包括自己, bn)也要同時prune掉。

* 在Design Architecure內，model的一個block，名稱所對應的Weight；

|#|name|meaning|code|weight shape|
|-|-|-|-|-|
|0|cnn.{i}.0|Depthwise Convolution Layer|nn.Conv2d(x, x, 3, 1, 1, group=x)|(x, 1, 3, 3)|
|1|cnn.{i}.1|Batch Normalization|nn.BatchNorm2d(x)|(x)|
|2||ReLU6|nn.ReLU6||
|3|cnn.{i}.3|Pointwise Convolution Layer|nn.Conv2d(x, y, 1),|(y, x, 1, 1)|
|4||MaxPooling|nn.MaxPool2d(2, 2, 0)||

In [ ]:
def network_slimming(old_model, new_model):
    params = old_model.state_dict()
    new_params = new_model.state_dict()
    
    # selected_idx: 每一層所選擇的neuron index
    selected_idx = []
    # 我們總共有7層CNN，因此逐一抓取選擇的neuron index們。
    for i in range(8):
        # 根據上表，我們要抓的gamma係數在cnn.{i}.1.weight內。
        importance = params[f'cnn.{i}.1.weight']
        # 抓取總共要篩選幾個neuron。
        old_dim = len(importance)
        new_dim = len(new_params[f'cnn.{i}.1.weight'])
        # 以Ranking做Index排序，較大的會在前面(descending=True)。
        ranking = torch.argsort(importance, descending=True)
        # 把篩選結果放入selected_idx中。
        selected_idx.append(ranking[:new_dim])

    now_processed = 1
    for (name, p1), (name2, p2) in zip(params.items(), new_params.items()):
        # 如果是cnn層，則移植參數。
        # 如果是FC層，或是該參數只有一個數字(例如batchnorm的tracenum等等資訊)，那麼就直接複製。
        if name.startswith('cnn') and p1.size() != torch.Size([]) and now_processed != len(selected_idx):
            # 當處理到Pointwise的weight時，讓now_processed+1，表示該層的移植已經完成。
            if name.startswith(f'cnn.{now_processed}.3'):
                now_processed += 1

            # 如果是pointwise，weight會被上一層的pruning和下一層的pruning所影響，因此需要特判。
            if name.endswith('3.weight'):
                # 如果是最後一層cnn，則輸出的neuron不需要prune掉。
                if len(selected_idx) == now_processed:
                    new_params[name] = p1[:,selected_idx[now_processed-1]]
                # 反之，就依照上層和下層所選擇的index進行移植。
                # 這裡需要注意的是Conv2d(x,y,1)的weight shape是(y,x,1,1)，順序是反的。
                else:
                    new_params[name] = p1[selected_idx[now_processed]][:,selected_idx[now_processed-1]]
            else:
                new_params[name] = p1[selected_idx[now_processed]]
        else:
            new_params[name] = p1

    # 讓新model load進被我們篩選過的parameters，並回傳new_model。        
    new_model.load_state_dict(new_params)
    return new_model


In [ ]:
# prune resnet18

def network_slimming(old_model, new_model):
    params = old_model.state_dict()
    new_params = new_model.state_dict()
    
    # selected_idx: 每一層所選擇的neuron index
    selected_idx = []
    # 我們總共有20層CNN，因此逐一抓取選擇的neuron index們。
    for i in range(20):
        # 根據上表，我們要抓的gamma係數在cnn.{i}.1.weight內。
        importance = params[f'cnn.{i}.1.weight']
        # 抓取總共要篩選幾個neuron。
        old_dim = len(importance)
        new_dim = len(new_params[f'cnn.{i}.1.weight'])
        # 以Ranking做Index排序，較大的會在前面(descending=True)。
        ranking = torch.argsort(importance, descending=True)
        # 把篩選結果放入selected_idx中。
        selected_idx.append(ranking[:new_dim])

    now_processed = 1
    for (name, p1), (name2, p2) in zip(params.items(), new_params.items()):
        # 如果是cnn層，則移植參數。
        # 如果是FC層，或是該參數只有一個數字(例如batchnorm的tracenum等等資訊)，那麼就直接複製。
        if name.startswith('cnn') and p1.size() != torch.Size([]) and now_processed != len(selected_idx):
            # 當處理到Pointwise的weight時，讓now_processed+1，表示該層的移植已經完成。
            if name.startswith(f'cnn.{now_processed}.3'):
                now_processed += 1

            # 如果是pointwise，weight會被上一層的pruning和下一層的pruning所影響，因此需要特判。
            if name.endswith('3.weight'):
                # 如果是最後一層cnn，則輸出的neuron不需要prune掉。
                if len(selected_idx) == now_processed:
                    new_params[name] = p1[:,selected_idx[now_processed-1]]
                # 反之，就依照上層和下層所選擇的index進行移植。
                # 這裡需要注意的是Conv2d(x,y,1)的weight shape是(y,x,1,1)，順序是反的。
                else:
                    new_params[name] = p1[selected_idx[now_processed]][:,selected_idx[now_processed-1]]
            else:
                new_params[name] = p1[selected_idx[now_processed]]
        else:
            new_params[name] = p1

    # 讓新model load進被我們篩選過的parameters，並回傳new_model。        
    new_model.load_state_dict(new_params)
    return new_model


# Data Processing

我們的Dataset使用的是跟Hw3 - CNN同樣的Dataset，因此這個區塊的Augmentation / Read Image大家參考就好。

如果有不會的話可以回去看Hw3的colab。

In [ ]:
import re
import torch
from glob import glob
from PIL import Image
import torchvision.transforms as transforms

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, folderName, transform=None):
        self.transform = transform
        self.data = []
        self.label = []

        for img_path in sorted(glob(folderName + '/*.jpg')):
            try:
                # Get classIdx by parsing image path
                class_idx = int(re.findall(re.compile(r'\d+'), img_path)[1])
            except:
                # if inference mode (there's no answer), class_idx default 0
                class_idx = 0
 
            image = Image.open(img_path)
            # Get File Descriptor
            image_fp = image.fp
            image.load()
            # Close File Descriptor (or it'll reach OPEN_MAX)
            image_fp.close()

            self.data.append(image)
            self.label.append(class_idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, self.label[idx]


trainTransform = transforms.Compose([
    transforms.RandomCrop(256, pad_if_needed=True, padding_mode='symmetric'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
testTransform = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])

def get_dataloader(mode='training', batch_size=32):

    assert mode in ['training', 'testing', 'validation']

    dataset = MyDataset(
        f'./food-11/{mode}',
        transform=trainTransform if mode == 'training' else testTransform)

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=(mode == 'training'))

    return dataloader


# Pre-processing

我們已經提供原始小model binary，架構是hw7_Architecture_Design.ipynb中的StudentNet。

In [ ]:
# get dataloader
train_dataloader = get_dataloader('training', batch_size=32)
print('finish train_dataloader')

valid_dataloader = get_dataloader('validation', batch_size=32)
print('finish valid_dataloader')

In [ ]:
# !gdown --id '12wtIa0WVRcpboQzhgRUJOpcXe23tgWUL' --output student_custom_small.bin
!gdown --id '1-BVZoTUkX0faW4sYk7L2qiZbo-uYdn0P' --output student_custom_small.bin #使用上去 kaggle 的

net = StudentNet().cuda()
net.load_state_dict(torch.load('student_custom_small.bin'))

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=1e-3)

In [ ]:
# 使用 pre-train teacher_resnet18.bin
!gdown --id '1B8ljdrxYXJsZv2vmTequdPOofp3VF3NN' --output teacher_resnet18.bin

net = models.resnet18(pretrained=False, num_classes=11).cuda() #把 teacher net load

net.load_state_dict(torch.load(f'./teacher_resnet18.bin'))
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=1e-3)

# Start Training

* 每次Prune rate是0.95，Prune完後會重新fine-tune 3 epochs。
* 其餘的步驟與你在做Hw3 - CNN的時候一樣。



In [ ]:
def get_parameter_number(net):
    # total_num = sum(p.numel() for p in net.parameters())
    trainable_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print('trainable parameters =', trainable_num)
    return trainable_num

In [ ]:
import time
from torchsummary import summary

def run_epoch(dataloader, update=True, alpha=0.5):
    total_num, total_hit, total_loss = 0, 0, 0
    for now_step, batch_data in enumerate(dataloader):
        # 清空 optimizer
        optimizer.zero_grad()
        # 處理 input
        inputs, labels = batch_data
        inputs = inputs.cuda()
        labels = labels.cuda()
  
        logits = net(inputs)
        loss = criterion(logits, labels)
        if update:
            loss.backward()
            optimizer.step()

        total_hit += torch.sum(torch.argmax(logits, dim=1) == labels).item()
        total_num += len(inputs)
        total_loss += loss.item() * len(inputs)

    return total_loss / total_num, total_hit / total_num


parameters_acc_095 = []
now_width_mult = 1

for i in range(5):
    # print(summary(net, input_size=(3, 128, 128)))
    print('for i =', i)
    now_width_mult *= 0.95
    new_net = StudentNet(width_mult=now_width_mult).cuda()
    # new_net = models.resnet18(pretrained=False, num_classes=11).cuda()
    params = net.state_dict()
    net = network_slimming(net, new_net)
    now_best_acc = 0
    for epoch in range(5):
        # print(summary(net, input_size=(3, 128, 128)))
        train_start_time = time.time()
        net.train()
        train_loss, train_acc = run_epoch(train_dataloader, update=True)
        net.eval()
        valid_loss, valid_acc = run_epoch(valid_dataloader, update=False)
        # 在每個width_mult的情況下，存下最好的model。
        if valid_acc > now_best_acc:
            now_best_acc = valid_acc
            torch.save(net.state_dict(), f'custom_small_rate_{now_width_mult}.bin')
            print('save model')
        print('rate {:6.4f} epoch {:>3d}: train loss: {:6.4f}, acc {:6.4f} valid loss: {:6.4f}, acc {:6.4f}'.format(now_width_mult, 
            epoch, train_loss, train_acc, valid_loss, valid_acc))
        print('epoch cost time =', time.time() - train_start_time)
        print('')
    parameters_acc_095.append([get_parameter_number(net),now_best_acc])
    print('----------------------------------------------------------------------------------------------')


In [ ]:
net_090 = StudentNet().cuda()
net_090.load_state_dict(torch.load('student_custom_small.bin'))

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net_090.parameters(), lr=1e-3)

In [ ]:
import time
from torchsummary import summary

def run_epoch(dataloader, update=True, alpha=0.5):
    total_num, total_hit, total_loss = 0, 0, 0
    for now_step, batch_data in enumerate(dataloader):
        # 清空 optimizer
        optimizer.zero_grad()
        # 處理 input
        inputs, labels = batch_data
        inputs = inputs.cuda()
        labels = labels.cuda()
  
        logits = net_090(inputs)
        loss = criterion(logits, labels)
        if update:
            loss.backward()
            optimizer.step()

        total_hit += torch.sum(torch.argmax(logits, dim=1) == labels).item()
        total_num += len(inputs)
        total_loss += loss.item() * len(inputs)

    return total_loss / total_num, total_hit / total_num


parameters_acc_090 = []
now_width_mult = 1

for i in range(5):
    # print(summary(net, input_size=(3, 128, 128)))
    print('for i =', i)
    now_width_mult *= 0.90
    new_net = StudentNet(width_mult=now_width_mult).cuda()
    # new_net = models.resnet18(pretrained=False, num_classes=11).cuda()
    params = net_090.state_dict()
    net_090 = network_slimming(net_090, new_net)
    now_best_acc = 0
    for epoch in range(5):
        # print(summary(net, input_size=(3, 128, 128)))
        train_start_time = time.time()
        net_090.train()
        train_loss, train_acc = run_epoch(train_dataloader, update=True)
        net_090.eval()
        valid_loss, valid_acc = run_epoch(valid_dataloader, update=False)
        # 在每個width_mult的情況下，存下最好的model。
        if valid_acc > now_best_acc:
            now_best_acc = valid_acc
            torch.save(net_090.state_dict(), f'custom_small_rate_{now_width_mult}.bin')
            print('save model')
        print('rate {:6.4f} epoch {:>3d}: train loss: {:6.4f}, acc {:6.4f} valid loss: {:6.4f}, acc {:6.4f}'.format(now_width_mult, 
            epoch, train_loss, train_acc, valid_loss, valid_acc))
        print('epoch cost time =', time.time() - train_start_time)
        print('')
    parameters_acc_090.append([get_parameter_number(net_090),now_best_acc])
    print('----------------------------------------------------------------------------------------------')


## check parameters

In [ ]:
from torchsummary import summary
summary(net, input_size=(3, 128, 128))

# from torchsummary import summary
# summary(net_090, input_size=(3, 128, 128))

## 畫出圖片

In [ ]:
parameters_1 = []
acc_1 = []
for i in range(len(parameters_acc)):
  parameters_1.append(parameters_acc[i][0])
  acc_1.append(parameters_acc[i][1])

parameters_2 = []
acc_2 = []
for i in range(len(parameters_acc_090)):
  parameters_2.append(parameters_acc_090[i][0])
  acc_2.append(parameters_acc_090[i][1])

In [ ]:
import matplotlib.pyplot as plt

plt.title('Comparison between the different pruning rate')
plt.plot(parameters_2, acc_2, label='pruning 0.9') #pruning 0.9
plt.plot(parameters_1, acc_1, label='pruning 0.95') #pruning 0.95

plt.xlabel('num of parameters')
plt.ylabel('val_acc')
plt.legend(loc='lower right')

plt.show()

# Inference

同Hw3，請參考該作業:)。

## testing

In [ ]:
test_dataloader = get_dataloader('testing', batch_size=32)
print('finish test_dataloader')

In [ ]:
import numpy as np
net.eval()
prediction = []

for now_step, batch_data in enumerate(test_dataloader):
    # 清空 optimizer
    optimizer.zero_grad()
    # 處理 input
    inputs, labels = batch_data
    inputs = inputs.cuda()
    # labels = labels.cuda() 
    with torch.no_grad():
        logits = net(inputs)
        test_label = np.argmax(logits.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)     

In [ ]:
# 丟到 hw7
from google.colab import files

#將結果寫入 csv 檔
with open("predict.csv", 'w') as f:
    f.write('Id,label\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))
#存到本機端
files.download('predict.csv')

In [ ]:
# 丟到 hw3
from google.colab import files

#將結果寫入 csv 檔
with open("predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))
#存到本機端
files.download('predict.csv')

# Q&A

有任何問題Network Compression的問題可以寄信到b05902127@ntu.edu.tw。

時間允許的話我會更新在這裡。